In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
from typing import Callable
from pathlib import Path
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, solve_bvp
from scipy.optimize import brentq
try:
    import jax
    import jax.numpy as jnp
    from jax import jacfwd
    JAX_AVAILABLE = True
except ImportError:
    JAX_AVAILABLE = False

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 8), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note(f"Environment initialized. JAX available: {JAX_AVAILABLE}")

# Part 2: Core Numerical Methods
## Chapter 2.8: Differential Equations and Dynamic Systems

### Table of Contents
1.  [Theory of Dynamic Systems](#1.-Theory-of-Dynamic-Systems)
    *   [1.1 Phase Diagrams and Nullclines](#1.1-Phase-Diagrams-and-Nullclines)
    *   [1.2 Stability Analysis via Linearization](#1.2-Stability-Analysis-via-Linearization)
    *   [1.3 Bifurcation Theory](#1.3-Bifurcation-Theory)
2.  [Application: The Ramsey-Cass-Koopmans (RCK) Model](#2.-Application:-The-Ramsey-Cass-Koopmans-(RCK)-Model)
    *   [2.1 The Model as an Optimal Control Problem](#2.1-The-Model-as-an-Optimal-Control-Problem)
    *   [2.2 Solving for the Saddle Path](#2.2-Solving-for-the-Saddle-Path)
3.  [Numerical Methods for Solving ODEs](#3.-Numerical-Methods-for-Solving-ODEs)
    *   [3.1 Runge-Kutta Methods and Stiffness](#3.1-Runge-Kutta-Methods-and-Stiffness)
    *   [3.2 Boundary Value Problems: Shooting vs. Relaxation](#3.2-Boundary-Value-Problems:-Shooting-vs.-Relaxation)
4.  [Chapter Summary](#4.-Chapter-Summary)
5.  [Exercises](#5.-Exercises)

### Introduction: Modeling Economic Dynamics
Many economic models are not static; they describe systems that evolve over time. The mathematical language for describing continuous change is the **differential equation**. A system of **Ordinary Differential Equations (ODEs)** describes the rate of change of a set of state variables as a function of the current state of those variables. In macroeconomics, finance, and growth theory, these systems are foundational.

This chapter provides a guide to analyzing these dynamic systems. We will cover:
1.  **Phase Diagrams & Stability:** The primary tools for qualitatively analyzing dynamic systems, including linearization and an introduction to **bifurcation theory**.
2.  **Optimal Control:** Formally setting up a dynamic optimization problem using the **Hamiltonian** to derive the necessary conditions that form the system of ODEs.
3.  **Numerical Solvers:** Understanding the different algorithms for solving **Initial Value Problems (IVPs)** and **Boundary Value Problems (BVPs)**, including the crucial concepts of stiffness and saddle path dynamics.
4.  **Applications:** Applying these tools to the canonical **Ramsey-Cass-Koopmans (RCK) growth model**.

### 1. Theory of Dynamic Systems

#### 1.1 Phase Diagrams and Nullclines
A **phase diagram** is the primary tool for visualizing the behavior of a two-dimensional dynamic system. The key is to find the **nullclines**—the lines where one of the variables is not changing—and then determine the direction of motion in the regions between them. The intersection of the nullclines gives the system's **steady states** or **fixed points**.

![Lotka-Volterra Phase Diagram](../images/png/lotka_volterra.png)
*Figure 1: The Lotka-Volterra phase diagram. The red and blue dashed lines are the nullclines, where the prey and predator populations, respectively, are constant. The black dot represents the steady state, and the colored lines show the cyclical dynamics of the system. Generated by `scripts/generate_images.py`.*

#### 1.2 Stability Analysis via Linearization
We can formally analyze the stability of a steady state by linearizing the system. The **Hartman-Grobman theorem** states that near a hyperbolic fixed point (where no eigenvalues have zero real part), the behavior of the nonlinear system is qualitatively the same as its linearization. The local dynamics are governed by the **Jacobian matrix** $J(\mathbf{z}^*)$, and its **eigenvalues** determine the local stability:
- **All $\text{Re}(\lambda_i) < 0$**: The steady state is locally stable (a sink).
- **All $\text{Re}(\lambda_i) > 0$**: The steady state is locally unstable (a source).
- **Some $\text{Re}(\lambda_i) > 0$ and some $\text{Re}(\lambda_i) < 0$**: The steady state is a **saddle point**.
- **Some $\text{Re}(\lambda_i) = 0$**: The steady state is a center (or non-hyperbolic), and linearization is inconclusive.

#### 1.3 Bifurcation Theory
**Bifurcation theory** studies how the qualitative nature of a system's solutions changes as a parameter is varied. A **bifurcation** occurs at a critical parameter value where the system's stability or the number of steady states changes.

A classic example is the **saddle-node bifurcation** in the 1D system $\dot{x} = r + x^2$. The steady states are given by $x^* = \pm \sqrt{-r}$.
- For $r < 0$, there are two steady states: one stable ($x^* = -\sqrt{-r}$) and one unstable ($x^* = +\sqrt{-r}$).
- For $r = 0$, there is one semi-stable steady state at $x^*=0$.
- For $r > 0$, there are no real steady states.

The point $r=0$ is the bifurcation point where the two steady states collide and annihilate each other.

In [ ]:
sec("Visualizing a Saddle-Node Bifurcation")
sys.path.append(str(Path.cwd().parent.parent / 'src'))
from plotting_utils import plot_bifurcation
plot_bifurcation()

### 2. Application: The Ramsey-Cass-Koopmans (RCK) Model

#### 2.1 The Model as an Optimal Control Problem
The RCK model is a cornerstone of modern macroeconomics. It is formally an **optimal control problem**. The social planner chooses the path of the control variable (consumption, $c_t$) to maximize an objective function (lifetime utility) subject to a differential equation governing the evolution of the state variable (capital, $k_t$).

Objective: $ \max_{c_t} \int_0^\infty e^{-(\rho-n)t} \frac{c_t^{1-\theta}-1}{1-\theta} dt $
Subject to: $ \dot{k}_t = k_t^\alpha - (n+g+\delta)k_t - c_t $

The standard way to solve this is to set up the **current-value Hamiltonian**:
$$ \mathcal{H}(k, c, \lambda) = \frac{c^{1-\theta}-1}{1-\theta} + \lambda [k^\alpha - (n+g+\delta)k - c] $$
The necessary first-order conditions for an optimum are:
1. $\frac{\partial \mathcal{H}}{\partial c} = 0 \implies c^{-\theta} = \lambda$ (Optimality condition for consumption)
2. $\dot{\lambda} = (\rho-n)\lambda - \frac{\partial \mathcal{H}}{\partial k} \implies \dot{\lambda} = (\rho-n)\lambda - \lambda[\alpha k^{\alpha-1} - (n+g+\delta)]$ (Co-state equation)
3. The original law of motion for capital, $\dot{k}$.

By differentiating the first condition with respect to time and substituting the other two, we can eliminate $\lambda$ and arrive at the two-dimensional system in $(k, c)$ that we solve numerically.

#### 2.2 Solving for the Saddle Path
The RCK system has a saddle point steady state. For any given initial capital stock $k_0$, there is a **unique** initial consumption level $c_0$ that places the economy on the **stable saddle path**—the one trajectory that converges to the long-run steady state. Finding this path is a **Boundary Value Problem (BVP)**.

The code below solves for the steady state and then uses the shooting method to find the saddle path.

In [ ]:
sec("Solving the RCK Model with the Shooting Method")

# --- 1. Define Model Parameters ---
params = {
    'alpha': 0.33,  # Capital share
    'delta': 0.05,  # Depreciation rate
    'rho': 0.02,    # Discount rate
    'theta': 2.0,   # CRRA coefficient
    'n': 0.01,      # Population growth rate
    'g': 0.02       # Technology growth rate
}

# --- 2. Define Model Functions with Docstrings ---
def rck_model(t, z, p):
    """Defines the system of ODEs for the RCK model.
    
    Parameters
    ----------
    t : float
        Time (required by solve_ivp, but not used in this autonomous system).
    z : array_like, shape (2,)
        State vector [k, c].
    p : dict
        Dictionary of model parameters.
        
    Returns
    -------
    list, shape (2,)
        The time derivatives [dk/dt, dc/dt].
    """
    # Input validation
    required_keys = ['alpha', 'delta', 'rho', 'theta', 'n', 'g']
    if not all(key in p for key in required_keys):
        raise ValueError(f"Missing one or more required parameters: {required_keys}")
    if not 0 < p['alpha'] < 1:
        raise ValueError("alpha must be in (0,1)")
    
    k, c = z
    if k < 0 or c < 0:
        # Return large penalty to prevent solver from exploring negative values
        return [1e6, 1e6]
        
    k_dot = k**p['alpha'] - (p['n'] + p['g'] + p['delta']) * k - c
    c_dot = c / p['theta'] * (p['alpha'] * k**(p['alpha'] - 1) - p['delta'] - p['rho'] - p['theta'] * p['g'])
    return [k_dot, c_dot]

def shooting_error(c0, k0, T, p, k_star, c_star):
    """Calculates the error for the shooting method.
    
    Solves the RCK model forward from an initial guess c0 and returns the 
    difference between the terminal consumption and the steady-state consumption.
    
    Returns
    -------
    float
        Error: c(T) - c_star.
    """
    sol = solve_ivp(rck_model, [0, T], [k0, c0], args=(p,), dense_output=True, t_eval=[T])
    c_T = sol.y[1, -1]
    return c_T - c_star

# --- 3. Solve for Steady State and Saddle Path ---
k_star = (params['alpha'] / (params['delta'] + params['rho'] + params['theta'] * params['g']))**(1 / (1 - params['alpha']))
c_star = k_star**params['alpha'] - (params['n'] + params['g'] + params['delta']) * k_star

k0 = 1.0
T_horizon = 200

c0_saddle = brentq(shooting_error, a=0.1, b=c_star * 1.2, args=(k0, T_horizon, params, k_star, c_star))
note(f"For k0={k0}, the unique initial consumption on the saddle path is c0={c0_saddle:.4f}")



### 3. Numerical Methods for Solving ODEs

#### 3.1 Runge-Kutta Methods and Stiffness
`scipy.integrate.solve_ivp` is the modern interface for solving Initial Value Problems (IVPs). A critical choice is the `method` argument. The default, `RK45`, is a **Runge-Kutta method**. These methods achieve high accuracy by evaluating the function at several intermediate points within a single step to cancel out lower-order error terms.

A system is **stiff** if it involves processes on vastly different time scales. For these, explicit solvers like `RK45` are forced to take tiny steps to maintain stability. **Implicit methods** (like `BDF` or `Radau`) are far superior for stiff problems.

#### 3.2 Boundary Value Problems: Shooting vs. Relaxation
There are two main approaches to solving BVPs like the RCK model:
1. **The Shooting Method:** This transforms the BVP into an IVP. We guess the missing initial condition ($c_0$), solve the IVP forward in time, and check if the terminal condition is met. We use a root-finder to iterate on our guess for $c_0$ until the error in the terminal condition is zero.
2. **Relaxation Methods (`solve_bvp`):** These methods are generally more robust. They work by discretizing the entire solution path and solving a large system of algebraic equations for all the points on the path simultaneously. `scipy.integrate.solve_bvp` uses this approach.

![RCK Model Phase Diagram](../images/png/rck_model.png)
*Figure 2: The RCK model phase diagram. The blue line is the k-nullcline and the red dashed line is the c-nullcline. The green line shows the unique stable saddle path that converges to the steady state (black dot). The magenta dashed lines show unstable paths that diverge. Generated by `scripts/generate_images.py`.*

### 4. Chapter Summary
- **Qualitative Analysis:** Phase diagrams, nullclines, and stability analysis via linearization of the Jacobian are essential tools for understanding a dynamic system's behavior without necessarily solving it.
- **Bifurcations:** The qualitative nature of a system can change dramatically as a parameter crosses a bifurcation point.
- **Optimal Control:** Many economic dynamics problems are optimal control problems. The Hamiltonian provides a framework for deriving the necessary conditions, which form a system of ODEs.
- **Saddle Path Stability:** Many economic models (like the RCK model) feature saddle path stability, where a unique path converges to the steady state. Finding this path is a Boundary Value Problem.
- **Numerical Solvers:** Use `solve_ivp` for Initial Value Problems, being mindful of stiffness. Use `solve_bvp` or the shooting method for Boundary Value Problems. `solve_bvp` is generally more robust.

### 5. Exercises

1.  **Comparative Dynamics (Impatience):** In the RCK model, how does the steady state change if households become more impatient (i.e., the discount rate `rho` increases from 0.02 to 0.04)? Resolve for the new steady state and use the shooting method to find the new saddle path. Plot the new nullclines and saddle path on the original phase diagram. Explain intuitively why the steady state moves in this direction.

2.  **Bifurcation Analysis:** For the system $\dot{x} = r x - x^3$ (a pitchfork bifurcation):
    a. Find the steady states as a function of the parameter $r$.
    b. Determine the stability of each steady state by analyzing the sign of the derivative $f'(x)$.
    c. Draw the bifurcation diagram, plotting the steady state values $x^*$ on the y-axis against the parameter $r$ on the x-axis. Use solid lines for stable steady states and dashed lines for unstable ones.

3.  **Lotka-Volterra with JAX:** Use `jax.jacfwd` to compute the Jacobian of the Lotka-Volterra system at its non-trivial steady state. What are the eigenvalues? What do they imply about the stability of the system? (Hint: The eigenvalues will be purely imaginary, indicating cycles).
    
4.  **A Predator-Prey Model with Logistic Growth**: A more realistic model has the prey exhibit logistic growth: $\dot{R} = a R(1 - R/K) - b RF$, $\dot{F} = -c F + d RF$. Here, K is the carrying capacity for the rabbits. Find the new nullclines and steady state and create a phase diagram. How do the dynamics differ from the standard Lotka-Volterra model? Is the steady state stable?

5.  **Solving the RCK with `solve_bvp`**: The RCK model is a BVP. The boundary conditions are $k(0) = k_0$ and a terminal condition that the system is at the steady state, $k(T) = k_{star}$. Formulate the problem for `scipy.integrate.solve_bvp` by defining a function for the ODEs and a function for the boundary condition residuals. Compare your result to the shooting method.